In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용 - 리눅스 사용할때
import scipy
import scipy.stats as stats

import os
import re
import time
import datetime as dt # 사용시 datetime.dt 로 사용 # 현재 시간 사용
from datetime import datetime # 사용시 함수로 바로 사용 #-> 시간 비교

In [2]:
### 한글 폰트 설치 - 윈도우 사용할 때
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform

path = 'c:/Windows/Fonts/malgun.ttf'  # 또는 나눔고딕: 'c:/Windows/Fonts/NanumGothic.ttf'
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)

In [3]:
# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

## 데이터 전처리

In [4]:
df_heart = pd.read_csv("../../dataset/heart_2020_cleaned.csv")
df_heart.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [5]:
# BMI 사분위수를 이용한 이상치 제거
Q1 = 24.030000
Q3 = 31.420000
IQR = Q3-Q1
print(f"(1) Q1-1.5*IQR 값 : {Q1-1.5*IQR}")
print(f"(2) Q3+1.5*IQR 값 : {Q3+1.5*IQR}")

condition_bmi = f"{Q1-1.5*IQR} <= BMI <= {Q3+1.5*IQR}"

(1) Q1-1.5*IQR 값 : 12.945
(2) Q3+1.5*IQR 값 : 42.505


In [6]:
# BMI 기준 정의
def classify_bmi(bmi):
    if bmi < 18.5:
        return '저체중'
    elif 18.5 <= bmi <= 24.9:
        return '정상 체중'
    elif 25 <= bmi <= 29.9:
        return '과체중'
    elif 30 <= bmi <= 34.9:
        return '경도 비만'
    elif 35 <= bmi <= 39.9:
        return '중도 비만'
    else:
        return '고도 비만'

# Sleeping 기준 정의
def classify_sleep(hours):
    if hours < 6:
        return '불면증'
    elif 6 <= hours <= 9:
        return '정상'
    else:
        return '과다 수면'

# 신체 건강 정도 범주화
def classify_physical(value):
    if value < 3.24:
        return '나쁨'
    else:
        return '좋음'

# 정신 건강 정도 범주화
def classify_mental(value):
    if value < 3.80:
        return '나쁨'
    else:
        return '좋음'

# 나이 범주화
def classify_age_category(age_range):
    if age_range in ['18-24']:
        return '청소년기'
    elif age_range in ['25-29', '30-34', '35-39', '40-44', '45-49']:
        return '청년기'
    elif age_range in ['50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80 or older']:
        return '노년기'

# bmi 이상치 제거(df_heart_stable 생성)
df_heart_stable = df_heart.query(condition_bmi)

df_heart_stable['BMI'] = df_heart_stable['BMI'].apply(classify_bmi)
df_heart_stable['SleepTime'] = df_heart_stable['SleepTime'].apply(classify_sleep)
df_heart_stable['PhysicalHealth'] = df_heart_stable['PhysicalHealth'].apply(classify_physical)
df_heart_stable['MentalHealth'] = df_heart_stable['MentalHealth'].apply(classify_mental)
df_heart_stable['AgeCategory'] = df_heart_stable['AgeCategory'].apply(classify_age_category)

In [9]:
df_heart_stable.head(10)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,저체중,Yes,No,No,나쁨,좋음,No,Female,노년기,White,Yes,Yes,Very good,불면증,Yes,No,Yes
1,No,정상 체중,No,No,Yes,나쁨,나쁨,No,Female,노년기,White,No,Yes,Very good,정상,No,No,No
2,No,과체중,Yes,No,No,좋음,좋음,No,Male,노년기,White,Yes,Yes,Fair,정상,Yes,No,No
3,No,정상 체중,No,No,No,나쁨,나쁨,No,Female,노년기,White,No,No,Good,정상,No,No,Yes
4,No,정상 체중,No,No,No,좋음,나쁨,Yes,Female,청년기,White,No,Yes,Very good,정상,No,No,No
5,Yes,과체중,Yes,No,No,좋음,나쁨,Yes,Female,노년기,Black,No,No,Fair,과다 수면,No,No,No
6,No,정상 체중,No,No,No,좋음,나쁨,No,Female,노년기,White,No,Yes,Fair,불면증,Yes,No,Yes
7,No,경도 비만,Yes,No,No,좋음,나쁨,Yes,Female,노년기,White,Yes,No,Good,정상,Yes,No,No
8,No,과체중,No,No,No,나쁨,나쁨,No,Female,노년기,White,"No, borderline diabetes",No,Fair,불면증,No,Yes,No
9,No,고도 비만,No,No,No,나쁨,나쁨,Yes,Male,노년기,White,No,Yes,Good,과다 수면,No,No,No


## 귀무가설 1 : BMI는 심장병과 관련성이 낮다.

In [14]:
from scipy.stats import chi2_contingency

# 교차 테이블 생성
contingency_table = pd.crosstab(df_heart_stable['BMI'], df_heart_stable['HeartDisease'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

# 결과 출력
print("카이제곱 통계량:", chi2)
print("p-value:", p)
print("자유도:", dof)
print("기대 빈도:\n", expected)

# p-value 해석
alpha = 0.05
if p < alpha:
    print("귀무가설을 기각합니다. 두 변수 간에 상관관계가 있습니다.")
else:
    print("귀무가설을 기각하지 않습니다. 두 변수 간에 상관관계가 없습니다.")

ValueError: too many values to unpack (expected 2)

In [15]:
contingency_table

HeartDisease,No,Yes
BMI,,
경도 비만,54185,6034
고도 비만,10483,1127
과체중,102745,9814
저체중,4670,395
정상 체중,88986,6152
중도 비만,22140,2668
